# Problem Steatment:

---

As some bullets are always sold together with firearms, your sales have different destination channels: 1) Together with Firearms (TF sales) and 2) Replacement Sales (RS), for bullets requested by local shops (to be sold separately). 
While you have precise estimates from firearm makers regarding the TF market bullets, you only have a few variables that help you forecast future RS sales (along with historical sales data since 1810). 

Your company supplies two brands of firearm makers: Remington and Henry Lever. Each company sells different handgun models which require custom-fitted bullets. 
Your job is to forecast the sales of each bullet model that is going through the RS channel, as well as the sales aggregate by handgun (RS only).
Together with historical sales data you are provided with firearm sales (time series) for both Remington and Henry Level models to help you with the task.

We would like you to return a 12-months ahead forecast with a methodology of your liking as well as a brief document describing the thinking process of your journey through the prairies of the Old West.
You can develop your code both using Python or R, and we ask you to attach the code to the csv you will provide as task solution.

In [1]:
!pip3 install tsfresh
!pip3 install PTable

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
from pandas import concat
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from prettytable import PrettyTable


## Data Loading & EDA 

In [0]:
# Loading Data From Drive
BASE_DIR = '/content/drive/My Drive/JB Solutions task/'
bullet_DF = pd.read_csv(os.path.join(BASE_DIR,'bullets_sales.csv'))
handgun_DF = pd.read_csv(os.path.join(BASE_DIR,'handgun_sales.csv'))

In [4]:
bullet_DF.head()

,Date,Maker,Model,Bullets,Channel,Quantity
0,1810-01-01,Remington,model_1,bullets_A,TF,0
1,1810-01-02,Remington,model_1,bullets_A,TF,2
2,1810-01-03,Remington,model_1,bullets_A,TF,3
3,1810-01-04,Remington,model_1,bullets_A,TF,2
4,1810-01-05,Remington,model_1,bullets_A,TF,3


In [5]:
bullet_DF.tail()

,Date,Maker,Model,Bullets,Channel,Quantity
30679,1816-12-27,NaN,NaN,bullets_B,RS,1243
30680,1816-12-28,NaN,NaN,bullets_B,RS,1384
30681,1816-12-29,NaN,NaN,bullets_B,RS,1245
30682,1816-12-30,NaN,NaN,bullets_B,RS,1473
30683,1816-12-31,NaN,NaN,bullets_B,RS,1246


In [6]:
handgun_DF.head()

,Date,Maker,Model,Quantity
0,1810-01-01,Remington,model_1,4
1,1810-01-02,Remington,model_1,6
2,1810-01-03,Remington,model_1,13
3,1810-01-04,Remington,model_1,23
4,1810-01-05,Remington,model_1,32


In [7]:
handgun_DF.tail()

,Date,Maker,Model,Quantity
15337,1816-12-27,Henry Lever,model_3,199
15338,1816-12-28,Henry Lever,model_3,196
15339,1816-12-29,Henry Lever,model_3,170
15340,1816-12-30,Henry Lever,model_3,158
15341,1816-12-31,Henry Lever,model_3,171


In [8]:
bullet_DF.describe()

,Quantity
count,30684.000000
mean,195.154087
std,320.229264
min,0.000000
25%,9.000000
50%,29.000000
75%,261.000000
max,3328.000000


In [9]:
handgun_DF.describe()

,Quantity
count,15342.000000
mean,142.466823
std,115.265709
min,0.000000
25%,54.000000
50%,111.000000
75%,201.000000
max,685.000000


# Observation

As we can see Bullet seals data has 6 columns (Data,Maker,

*   As we can see Bullet seals data has 6 columns (Data,Maker,Model,Bullets,Channel,Quantity)
* And handgun seals data has 4 columns (Date,Maker,Model,Quantity)

*   Distribution of quantity in bullet sales has max 3328 bullets sold and min 0 bullets sold
*   Distribution of quantity in handgun sales has max 685 bullets sold and min 0 bullets sold


*   And in both the data freams we have data from 1810-01-01 to 31-12-1816




# Check for the missing values in Data fream

In [0]:
def percent_missing(df):
  percent_missing = df.isnull().sum() * 100 / len(df)
  DF = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
  print(DF)

In [11]:
percent_missing(bullet_DF)

         column_name  percent_missing
Date            Date              0.0
Maker          Maker             50.0
Model          Model             50.0
Bullets      Bullets              0.0
Channel      Channel              0.0
Quantity    Quantity              0.0


In [12]:
percent_missing(handgun_DF)

         column_name  percent_missing
Date            Date              0.0
Maker          Maker              0.0
Model          Model              0.0
Quantity    Quantity              0.0


# Check for the Outliers values in Data fream

In [13]:
fig = px.box(bullet_DF, y="Quantity")
fig.show()

In [14]:
bullet_DF['Quantity'].quantile([0.90,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,1.0]) 

0.90     613.00
0.91     652.00
0.92     700.00
0.93     752.00
0.94     812.00
0.95     885.00
0.96     969.00
0.97    1049.51
0.98    1162.00
0.99    1327.17
1.00    3328.00
Name: Quantity, dtype: float64

In [15]:
fig = px.box(handgun_DF, y="Quantity")
fig.show()

In [16]:
handgun_DF['Quantity'].quantile([0.90,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,1.0]) 

0.90    327.9
0.91    338.0
0.92    349.0
0.93    361.0
0.94    371.0
0.95    383.0
0.96    398.0
0.97    412.0
0.98    428.0
0.99    456.0
1.00    685.0
Name: Quantity, dtype: float64

# Observation:


*   Now as we can see that in the bullet sales there are 50% of datasets with missing value but think again as it is menstioned in problem steatment it self that the bullet sales data hase 2 channel of seals one with Firearms and other one without Fireamrs and these 50% values are of channel 2 so we will no going to remove them or fill them we will use in its original form only


*   And From the view of outliers we can not see any values which is non realistic given all values looks authentic.




In [17]:
fig = px.line(bullet_DF, x='Date', y='Quantity', color='Bullets',line_group="Bullets")
fig.show()

In [18]:
fig = px.line(handgun_DF, x='Date', y='Quantity', color='Maker')
fig.show()

In [19]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=handgun_DF.Date, y=handgun_DF['Quantity'][handgun_DF['Maker']=='Remington'], name="Remington",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=handgun_DF.Date, y=handgun_DF['Quantity'][handgun_DF['Maker']=='Henry Lever'], name="Henry Lever",
                         line_color='dimgray'))

fig.update_layout(title_text='Time Series with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()

# Observations:


*   As we can see above we have bunch of plots for the target valiables which in our case is Quantity of bullet sales and handgun sales
*   From the very first plot it is clear that the sell of bullet A is higher than other 2 type Decending order is Bullet A > Bullet B > Bullet C 


*   And rest 2 plots are for the sales of handguns:
*   From which we can observe that for the period of 1810 to 1812 Remington's sell was higher than Henry Lever


*   But for period of 1812 to mid 1813 Henry Lever has higher sell than Remington 
*   And again after mid 1813 Remington take over the sell of Hanry Lever

Note : 3rd plot is Rangeslider interactive plot you can easy slid for the requierd range of time period to objesrve the behaviour of sell 







### Note: Now as we are asked to forecast the sales of Bullets in RS channels only we will filter our data on that basis and work worth thos rows which has rs channle only



---


# Performance matrix
As we can see that there is no outliers and this is a regression model so we will use 'square error' as performance matrix here

---






# Bullet Sales Forecast

In [0]:
#Filtering Bullet sales Data
bullet_RS_DF = bullet_DF[bullet_DF['Channel']=='RS']

In [21]:
bullet_RS_DF.head()

,Date,Maker,Model,Bullets,Channel,Quantity
15342,1810-01-01,NaN,NaN,bullets_A,RS,87
15343,1810-01-02,NaN,NaN,bullets_A,RS,147
15344,1810-01-03,NaN,NaN,bullets_A,RS,291
15345,1810-01-04,NaN,NaN,bullets_A,RS,237
15346,1810-01-05,NaN,NaN,bullets_A,RS,145


In [0]:
bullet_RS_DF = bullet_RS_DF.drop(['Maker', 'Model','Channel'], axis=1)

In [23]:
bullet_RS_DF.shape

(15342, 3)

# Train Test Split

In [0]:
train_bullet_RS_DF = bullet_RS_DF[bullet_RS_DF['Date'] < '1816-01-01']
test_bullet_RS_DF = bullet_RS_DF[bullet_RS_DF['Date'] >= '1816-01-01']

In [25]:
print(train_bullet_RS_DF.shape)
print(test_bullet_RS_DF.shape)

(13146, 3)
(2196, 3)


In [26]:
train_bullet_RS_DF.head()

,Date,Bullets,Quantity
15342,1810-01-01,bullets_A,87
15343,1810-01-02,bullets_A,147
15344,1810-01-03,bullets_A,291
15345,1810-01-04,bullets_A,237
15346,1810-01-05,bullets_A,145


# Featurization

In [0]:
train_bullet_RS_DF = train_bullet_RS_DF.groupby(['Date','Bullets'],as_index=False).agg({'Quantity' : sum})  
test_bullet_RS_DF = test_bullet_RS_DF.groupby(['Date','Bullets'],as_index=False).agg({'Quantity' : sum})  

In [28]:
print("train_bullet_RS_DF_new Shape",train_bullet_RS_DF.shape)
print("test_bullet_RS_DF_new Shape",test_bullet_RS_DF.shape)

train_bullet_RS_DF_new Shape (6573, 3)
test_bullet_RS_DF_new Shape (1098, 3)


In [29]:
test_bullet_RS_DF.head()

,Date,Bullets,Quantity
0,1816-01-01,bullets_A,1212
1,1816-01-01,bullets_B,1509
2,1816-01-01,bullets_C,247
3,1816-01-02,bullets_A,1207
4,1816-01-02,bullets_B,1274


In [0]:
train_bullet_RS_DF_new = concat([train_bullet_RS_DF, train_bullet_RS_DF['Quantity'].shift(1),train_bullet_RS_DF['Quantity'].shift(2),
                       train_bullet_RS_DF['Quantity'].shift(3),train_bullet_RS_DF['Quantity'].shift(4),
                       train_bullet_RS_DF['Quantity'].shift(5)], axis=1).sort_index()
train_bullet_RS_DF_new.columns = ['Date', 'Bullets','Quantity','t-1','t-2','t-3','t-4','t-5']

In [31]:
train_bullet_RS_DF.head()

,Date,Bullets,Quantity
0,1810-01-01,bullets_A,553
1,1810-01-01,bullets_B,394
2,1810-01-01,bullets_C,39
3,1810-01-02,bullets_A,1047
4,1810-01-02,bullets_B,1012


In [0]:
test_bullet_RS_DF_new = concat([test_bullet_RS_DF, test_bullet_RS_DF['Quantity'].shift(1),test_bullet_RS_DF['Quantity'].shift(2),
                       test_bullet_RS_DF['Quantity'].shift(3),test_bullet_RS_DF['Quantity'].shift(4),
                       test_bullet_RS_DF['Quantity'].shift(5)], axis=1).sort_index()
test_bullet_RS_DF_new.columns = ['Date', 'Bullets','Quantity','t-1','t-2','t-3','t-4','t-5']

In [33]:
test_bullet_RS_DF_new.head()

,Date,Bullets,Quantity,t-1,t-2,t-3,t-4,t-5
0,1816-01-01,bullets_A,1212,NaN,NaN,NaN,NaN,NaN
1,1816-01-01,bullets_B,1509,1212.0,NaN,NaN,NaN,NaN
2,1816-01-01,bullets_C,247,1509.0,1212.0,NaN,NaN,NaN
3,1816-01-02,bullets_A,1207,247.0,1509.0,1212.0,NaN,NaN
4,1816-01-02,bullets_B,1274,1207.0,247.0,1509.0,1212.0,NaN


In [0]:
train_bullet_RS_DF_new['rolling_mean'] = train_bullet_RS_DF_new['Quantity'].rolling(window=5).mean()
test_bullet_RS_DF_new['rolling_mean'] = test_bullet_RS_DF_new['Quantity'].rolling(window=5).mean()

In [0]:
train_bullet_RS_DF_new['expanding_mean'] = train_bullet_RS_DF_new['Quantity'].expanding(5).mean()
test_bullet_RS_DF_new['expanding_mean'] = test_bullet_RS_DF_new['Quantity'].expanding(5).mean()


In [0]:
train_bullet_RS_DF_new = concat([train_bullet_RS_DF_new,pd.get_dummies(train_bullet_RS_DF_new['Bullets'])],axis=1)
test_bullet_RS_DF_new = concat([test_bullet_RS_DF_new,pd.get_dummies(test_bullet_RS_DF_new['Bullets'])],axis=1)

In [0]:
train_bullet_RS_DF_new.drop(['Bullets'],inplace = True,axis=1)
test_bullet_RS_DF_new.drop(['Bullets'],inplace = True,axis=1)

In [38]:
train_bullet_RS_DF_new.head()

,Date,Quantity,t-1,t-2,t-3,t-4,t-5,rolling_mean,expanding_mean,bullets_A,bullets_B,bullets_C
0,1810-01-01,553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
1,1810-01-01,394,553.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2,1810-01-01,39,394.0,553.0,NaN,NaN,NaN,NaN,NaN,0,0,1
3,1810-01-02,1047,39.0,394.0,553.0,NaN,NaN,NaN,NaN,1,0,0
4,1810-01-02,1012,1047.0,39.0,394.0,553.0,NaN,609.0,609.0,0,1,0


In [39]:
test_bullet_RS_DF_new.head()

,Date,Quantity,t-1,t-2,t-3,t-4,t-5,rolling_mean,expanding_mean,bullets_A,bullets_B,bullets_C
0,1816-01-01,1212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
1,1816-01-01,1509,1212.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2,1816-01-01,247,1509.0,1212.0,NaN,NaN,NaN,NaN,NaN,0,0,1
3,1816-01-02,1207,247.0,1509.0,1212.0,NaN,NaN,NaN,NaN,1,0,0
4,1816-01-02,1274,1207.0,247.0,1509.0,1212.0,NaN,1089.8,1089.8,0,1,0


In [0]:
X_train = train_bullet_RS_DF_new.drop(['Quantity','Date'],axis=1)
X_train = X_train.fillna(0)
X_test = test_bullet_RS_DF_new.drop(['Quantity','Date'],axis=1)
X_test = X_test.fillna(0)


In [0]:
y_train = train_bullet_RS_DF_new['Quantity']
y_test = test_bullet_RS_DF_new['Quantity']

# Prediction Model

In [42]:
# Parameter tunning

param_dist = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.01,0.05,0.1,0.3,1],
 
 }

parameter_tunning_bullets = RandomizedSearchCV(AdaBoostRegressor(),
param_distributions = param_dist,
 cv=3,
 n_iter = 10,
 n_jobs=-1)

parameter_tunning_bullets.fit(X_train, y_train)

# Best parameter
parameter_tunning_bullets.best_params_

{'learning_rate': 0.05, 'n_estimators': 100}

In [43]:
# Tunned Regerssor 
regresor_bullets = AdaBoostRegressor(random_state=0, n_estimators=50,learning_rate=0.1,loss = 'square')
regresor_bullets.fit(X_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='square',
                  n_estimators=50, random_state=0)

In [0]:
y_pred_bullets_sales = regresor_bullets.predict(X_test)

In [45]:
train_score_bullets = regresor_bullets.score(X_train, y_train)
print("train_score",train_score_bullets)
test_score_bullets = regresor_bullets.score(X_test, y_test)
print("test_score",test_score_bullets)

train_score 0.915417571636775
test_score 0.8904124853691987


# Plotting prediction with actual value

In [0]:
frame_bullets = { 'Date': test_bullet_RS_DF['Date'], 'Bullets': test_bullet_RS_DF['Bullets'], 'true_quantity': test_bullet_RS_DF['Quantity'], 'pred_quantity': y_pred_bullets_sales } 
  
result_bullets = pd.DataFrame(frame_bullets) 

In [47]:
result_bullets.head()

,Date,Bullets,true_quantity,pred_quantity
0,1816-01-01,bullets_A,1212,129.126876
1,1816-01-01,bullets_B,1509,129.126876
2,1816-01-01,bullets_C,247,129.126876
3,1816-01-02,bullets_A,1207,1147.740823
4,1816-01-02,bullets_B,1274,1502.362832


In [48]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=result_bullets.Date, y=result_bullets['true_quantity'], name="true_quantity",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=result_bullets.Date, y=result_bullets['pred_quantity'], name="pred_quantity",
                         line_color='dimgray'))

fig.update_layout(title_text='Time Series with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()



---



---



---



---



# Handgun Sales Forecast

# Train Test Split

In [0]:
train_handgun_DF = handgun_DF[handgun_DF['Date'] < '1816-01-01']
test_handgun_DF = handgun_DF[handgun_DF['Date'] >= '1816-01-01']

In [50]:
print(train_handgun_DF.shape)
print(test_handgun_DF.shape)

(13146, 4)
(2196, 4)


In [51]:
train_handgun_DF.head()

,Date,Maker,Model,Quantity
0,1810-01-01,Remington,model_1,4
1,1810-01-02,Remington,model_1,6
2,1810-01-03,Remington,model_1,13
3,1810-01-04,Remington,model_1,23
4,1810-01-05,Remington,model_1,32


In [0]:
train_handgun_DF_new = concat([train_handgun_DF, train_handgun_DF['Quantity'].shift(1),train_handgun_DF['Quantity'].shift(2),
                       train_handgun_DF['Quantity'].shift(3),train_handgun_DF['Quantity'].shift(4),
                       train_handgun_DF['Quantity'].shift(5)], axis=1).sort_index()
train_handgun_DF_new.columns = ['Date', 'Maker','Model','Quantity','t-1','t-2','t-3','t-4','t-5']

In [0]:
test_handgun_DF_new = concat([test_handgun_DF, test_handgun_DF['Quantity'].shift(1),test_handgun_DF['Quantity'].shift(2),
                       test_handgun_DF['Quantity'].shift(3),test_handgun_DF['Quantity'].shift(4),
                       test_handgun_DF['Quantity'].shift(5)], axis=1).sort_index()
test_handgun_DF_new.columns = ['Date', 'Maker','Model','Quantity','t-1','t-2','t-3','t-4','t-5']

In [0]:
train_handgun_DF_new['rolling_mean'] = train_handgun_DF_new['Quantity'].rolling(window=5).mean()
test_handgun_DF_new['rolling_mean'] = test_handgun_DF_new['Quantity'].rolling(window=5).mean()

In [0]:
train_handgun_DF_new['expanding_mean'] = train_handgun_DF_new['Quantity'].expanding(5).mean()
test_handgun_DF_new['expanding_mean'] = test_handgun_DF_new['Quantity'].expanding(5).mean()

In [0]:
train_handgun_DF_new = concat([train_handgun_DF_new,pd.get_dummies(train_handgun_DF_new['Maker'])],axis=1)
test_handgun_DF_new = concat([test_handgun_DF_new,pd.get_dummies(test_handgun_DF_new['Maker'])],axis=1)

In [0]:
train_handgun_DF_new = concat([train_handgun_DF_new,pd.get_dummies(train_handgun_DF_new['Model'])],axis=1)
test_handgun_DF_new = concat([test_handgun_DF_new,pd.get_dummies(test_handgun_DF_new['Model'])],axis=1)

In [0]:
train_handgun_DF_new.drop(['Maker','Model'],inplace = True,axis=1)
test_handgun_DF_new.drop(['Maker','Model'],inplace = True,axis=1)

In [59]:
train_handgun_DF_new.head()

,Date,Quantity,t-1,t-2,t-3,t-4,t-5,rolling_mean,expanding_mean,Henry Lever,Remington,model_1,model_2,model_3
0,1810-01-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,0,0
1,1810-01-02,6,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,0,0
2,1810-01-03,13,6.0,4.0,NaN,NaN,NaN,NaN,NaN,0,1,1,0,0
3,1810-01-04,23,13.0,6.0,4.0,NaN,NaN,NaN,NaN,0,1,1,0,0
4,1810-01-05,32,23.0,13.0,6.0,4.0,NaN,15.6,15.6,0,1,1,0,0


In [0]:
X_train = train_handgun_DF_new.drop(['Quantity','Date'],axis=1)
X_train = X_train.fillna(0)
X_test = test_handgun_DF_new.drop(['Quantity','Date'],axis=1)
X_test = X_test.fillna(0)


In [0]:
y_train = train_handgun_DF_new['Quantity']
y_test = test_handgun_DF_new['Quantity']

# Prediction Model

In [62]:
# Parameter tunning

param_dist = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.01,0.05,0.1,0.3,1],
 
 }

parameter_tunning_handgun = RandomizedSearchCV(AdaBoostRegressor(),
param_distributions = param_dist,
 cv=3,
 n_iter = 10,
 n_jobs=-1)

parameter_tunning_handgun.fit(X_train, y_train)

# Best parameter
parameter_tunning_handgun.best_params_

{'learning_rate': 0.05, 'n_estimators': 100}

In [63]:
# Tunned Regerssor 
regresor_handgun = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=0.05,loss = 'square')
regresor_handgun.fit(X_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.05, loss='square',
                  n_estimators=100, random_state=0)

In [0]:
y_pred_handgun_sales = regresor_handgun.predict(X_test)

In [65]:
score = regresor_handgun.score(X_test, y_test)
print(score)

0.9456263303272828


In [66]:
train_score_handgun = regresor_handgun.score(X_train, y_train)
print("train_score",train_score_handgun)
test_score_handgun = regresor_handgun.score(X_test, y_test)
print("test_score",test_score_handgun)

train_score 0.9729731320604614
test_score 0.9456263303272828


# Plotting Prediction

In [0]:
frame_handgun = { 'Date': test_handgun_DF['Date'], 'Maker': test_handgun_DF['Maker'], 'Model': test_handgun_DF['Model'] ,'true_quantity': test_handgun_DF['Quantity'], 'pred_quantity': y_pred_handgun_sales } 
  
result_handgun = pd.DataFrame(frame_handgun) 

In [68]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=result_handgun.Date, y=result_handgun['true_quantity'][result_handgun['Maker']=='Remington'], name="true_quantity_Remington",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=result_handgun.Date, y=result_handgun['pred_quantity'][result_handgun['Maker']=='Remington'], name="pred_quantity_Remington",
                         line_color='dimgray'))

fig.update_layout(title_text='Time Series with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()

In [69]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=result_handgun.Date, y=result_handgun['true_quantity'][result_handgun['Maker']=='Henry Lever'], name="true_quantity_Henry Lever",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=result_handgun.Date, y=result_handgun['pred_quantity'][result_handgun['Maker']=='Henry Lever'], name="pred_quantity_Henry Lever",
                         line_color='dimgray'))

fig.update_layout(title_text='Time Series with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()



---



---



---



---



# Result Table:

In [71]:
x = PrettyTable()
x.field_names = ["Gun/Bullet", "Algo", "Train Loss", "Test Loss"]
x.add_row(["Bullet", "AdaBoostRegressor",train_score_bullets ,train_score_bullets ])
x.add_row(["Handgun", "AdaBoostRegressor",train_score_handgun ,train_score_handgun ])
print(x)

+------------+-------------------+--------------------+--------------------+
| Gun/Bullet |        Algo       |     Train Loss     |     Test Loss      |
+------------+-------------------+--------------------+--------------------+
|   Bullet   | AdaBoostRegressor | 0.915417571636775  | 0.915417571636775  |
|  Handgun   | AdaBoostRegressor | 0.9729731320604614 | 0.9729731320604614 |
+------------+-------------------+--------------------+--------------------+


---


# Improvement

---

*   Above is the very firstcut solution to given problem
*   We can Improve that by adding more features to data
*   We can also add FS channel data to help predicting RS channel sales
*   We can apply advance technique and algos ex. for better featurization of data we can use deep learing to extract features from time series data
*   We can allso apply LSTM layers as this is time series dataset
